# Buidling Transition Matrices from Resume Data

Our preprocessed resume data contains job experiences of 62 million workers, each with at least 2 job experiences. Using this data, we build matrices of transitions and transition probabilities. 

Due to our large dataset, we use Dask, an open-source Python library for parallel computing, and run it on a local cluster set up on our local machine. 

In [ ]:
from dask.distributed import Client

client = Client(n_workers=7, threads_per_worker=3, memory_limit="15GB")
client

c:\Users\ewp\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:144: UserWarning: Creating scratch directories is taking a surprisingly long time. (3.92s) This is often due to running workers on a network file system. Consider specifying a local-directory to point workers to write scratch data to a local disk.
  next(self.gen)


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 7
Total threads: 21,Total memory: 97.79 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:59739,Workers: 7
Dashboard: http://127.0.0.1:8787/status,Total threads: 21
Started: Just now,Total memory: 97.79 GiB
Comm: tcp://127.0.0.1:59786,Total threads: 3
Dashboard: http://127.0.0.1:59787/status,Memory: 13.97 GiB
Nanny: tcp://127.0.0.1:59743,


In [2]:
import dask.dataframe as dd
from dask import delayed
import pandas as pd
from tqdm.auto import tqdm

Convert to Parquet and use Parquet! 

In [3]:
df = dd.read_csv("data/raw/Lightcast_jobs_filtered_v4_wo_missing_start_date.csv").iloc[:,1:].set_index("ID",inplace = True)

In [4]:
df.to_parquet("Lightcast_jobs_filtered_v4_wo_missing_start_date.parquet")

2024-06-06 23:48:40,853 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 4a791e4240357f895dcebdc0e3c7615f initialized by task ('shuffle-transfer-4a791e4240357f895dcebdc0e3c7615f', 536) executed on worker tcp://127.0.0.1:59776
2024-06-07 00:00:28,988 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 4a791e4240357f895dcebdc0e3c7615f deactivated due to stimulus 'task-finished-1717732828.8831992'
2024-06-07 00:00:42,654 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 6bebbd906ae414011cc3e04db3037424 initialized by task ('shuffle-transfer-6bebbd906ae414011cc3e04db3037424', 40) executed on worker tcp://127.0.0.1:59786
2024-06-07 00:21:45,425 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle fc0332b088fa2bf7c3e76a6fc451ccf8 initialized by task ('shuffle-transfer-fc0332b088fa2bf7c3e76a6fc451ccf8', 438) executed on worker tcp://127.0.0.1:59786
2024-06-07 08:29:58,055 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 6bebbd906ae414011cc3e04db3

In [5]:
ddf = dd.read_parquet("Lightcast_jobs_filtered_v4_wo_missing_start_date.parquet")

### Single Hop

In [1]:
def singlehopcount(group):
    # Update list
    updates2 = []

    group = group.reset_index(drop=True)

    # Go through the experiences of each ID 
    for i in range(len(group) - 1):
        # Extract current job info
        current_st = group.loc[i, "START_DATE"]
        current_end = group.loc[i, "END_DATE"]
        current_ONET = group.loc[i, "ONET"]
        
        if current_st == current_end: 
            first_hop_date = 0 

            for j in range(i + 1, len(group)):
                next_st = group.loc[j, "START_DATE"]
                next_end = group.loc[j, "END_DATE"]
                next_ONET = group.loc[j, "ONET"]

                # Note: When I am a dot, don't count to the following rows with the same start date
                '''
                if next_st == current_end and next_end != current_end: 
                    # Count hop: 
                    update_info = {
                                'current_ONET': current_ONET,  # The ONET code of the current job experience
                                'next_ONET': next_ONET,  # The ONET code of the next or overlapping job
                                'update_value': 1  # Assuming a simple count increment for each hop
                                }
                    updates2.append(update_info)
                ''' 

                # Only count the Non-overlapping component: 
                if next_st > current_end:  
                    # Date of Immediate Next Job 
                    if first_hop_date == 0: 
                        # Count hop 
                        first_hop_date = group.loc[j, "START_DATE"]   # Record first hop, but skip this count 
                        update_info = {
                                'current_ONET': current_ONET,  # The ONET code of the current job experience
                                'next_ONET': next_ONET,  # The ONET code of the next or overlapping job
                                'update_value': 1  # Assuming a simple count increment for each hop
                                }
                        updates2.append(update_info)
                    # Count all other jobs that have the same dat as the immediate next hop 
                    elif next_st==first_hop_date:  
                        update_info = {
                            'current_ONET': current_ONET,  # The ONET code of the current job experience
                            'next_ONET': next_ONET,  # The ONET code of the next or overlapping job
                            'update_value': 1  # Assuming a simple count increment for each hop
                            }
                        updates2.append(update_info)

        elif current_st != current_end: 
            first_hop_date = 0 

            for j in range(i + 1, len(group)):
                next_st = group.loc[j, "START_DATE"]
                next_end = group.loc[j, "END_DATE"]
                next_ONET = group.loc[j, "ONET"]

                # Condition for Overlapping: Next Start < Current End 
                # Only count the overlapping that have at least a month difference for the starting date
                if current_st < next_st < current_end: 
                    # Count hop: 
                    update_info = {
                                'current_ONET': current_ONET,  # The ONET code of the current job experience
                                'next_ONET': next_ONET,  # The ONET code of the next or overlapping job
                                'update_value': 1  # Assuming a simple count increment for each hop
                                }
                    updates2.append(update_info)

                # Non-overlapping component: 
                elif next_st >= current_end:
                      
                    # Date of Immediate Next Job 
                    if first_hop_date == 0: 
                        # Count hop 
                        first_hop_date = group.loc[j, "START_DATE"]   # Record first hop, but skip this count 

                        update_info = {
                                'current_ONET': current_ONET,  # The ONET code of the current job experience
                                'next_ONET': next_ONET,  # The ONET code of the next or overlapping job
                                'update_value': 1  # Assuming a simple count increment for each hop
                                }
                        updates2.append(update_info)

                    # Count all other jobs that have the same dat as the immediate next hop 
                    elif next_st==first_hop_date:  
                        update_info = {
                            'current_ONET': current_ONET,  # The ONET code of the current job experience
                            'next_ONET': next_ONET,  # The ONET code of the next or overlapping job
                            'update_value': 1  # Assuming a simple count increment for each hop
                            }
                        updates2.append(update_info)
   
    return pd.DataFrame(updates2)

In [7]:
result = ddf.groupby('ID').apply(singlehopcount, meta={'current_ONET': 'object', 'next_ONET': 'object', 'update_value': 'int'})

In [8]:
aggregated_ddf = result.groupby(['current_ONET', 'next_ONET']).agg({'update_value': 'sum'}).reset_index()


In [9]:
aggregated_ddf1 = aggregated_ddf.compute()

In [10]:
aggregated_ddf1

,current_ONET,next_ONET,update_value
0,41-9022.00,21-1093.00,7197.0
1,11-9039.00,41-3091.00,99.0
2,43-6014.00,43-4051.00,84567.0
3,99-9999.00,41-9031.00,3606.0
4,25-3099.00,21-1092.00,369.0
...,...,...,...
15087,29-1031.00,47-4041.00,4.0
15088,51-4051.00,25-2059.00,1.0
15089,49-2011.00,17-2011.00,1.0
15090,17-2199.05,33-9011.00,1.0


In [11]:
job_hop_count_df = pd.read_csv("data/raw/job_hop_count_empty.csv",index_col=0)
import numpy as np

In [12]:
aggregated_ddf1.to_parquet("Transitions_v5_1hop.parquet")
aggregated_ddf1.to_csv("Transitions_v5_1hop.csv")

piv = aggregated_ddf1.pivot_table(index='current_ONET', columns='next_ONET', values='update_value', fill_value=0)
all_labels = piv.index.union(job_hop_count_df.index)
pivot_table = piv.reindex(index=all_labels, columns=all_labels, fill_value=0)
pivot_table.to_csv("Transition_Matrix_v5_1hop.csv")

trans_prob = pivot_table.div(pivot_table.sum(axis = 1), axis=0).replace(np.nan, 0)
trans_prob.to_csv("Transition_Probab_Matrix_v5_1hop.csv")

In [13]:
trans_prob_backward = pivot_table.div(pivot_table.sum(axis = 0), axis=1).replace(np.nan, 0)

In [14]:
trans_prob_backward.to_csv("Bkwd_Transition_Probab_Matrix_v5_1hop.csv")

### Second Hops

In [9]:
def multihopcount(group):
    # Update list
    updates2 = []

    group = group.reset_index(drop=True)
    
    #display(group)

    # Go through the experiences of each ID 
    for i in range(len(group) - 1):
        # Extract current job info
        current_st = group.loc[i, "START_DATE"]
        current_end = group.loc[i, "END_DATE"]
        current_ONET = group.loc[i, "ONET"]
        
        if current_st == current_end: 
            first_hop_date = 0 
            second_hop_date = 0 

            for j in range(i + 1, len(group)):
                next_st = group.loc[j, "START_DATE"]
                next_end = group.loc[j, "END_DATE"]
                next_ONET = group.loc[j, "ONET"]

                if next_st > current_end:  # No overlaps 
                    if first_hop_date == 0: 
                        first_hop_date = group.loc[j, "START_DATE"]   # Record first hop, but skip this count 
                    
                    if next_st>first_hop_date:   # Next hop 
                        if (second_hop_date == 0) or (second_hop_date == next_st):
                            update_info = {
                                'current_ONET': current_ONET,  # The ONET code of the current job experience
                                'next_ONET': next_ONET,  # The ONET code of the next or overlapping job
                                'update_value': 1  # Assuming a simple count increment for each hop
                                }
                            updates2.append(update_info)

                            second_hop_date = next_st   # Record the start date of the second hop 


        elif current_st != current_end: 
            first_hop_date = 0 
            second_hop_date = 0 

            for j in range(i + 1, len(group)):
                next_st = group.loc[j, "START_DATE"]
                next_end = group.loc[j, "END_DATE"]
                next_ONET = group.loc[j, "ONET"]

                if next_st >= current_end:  # No overlaps 
                    if first_hop_date == 0: 
                        first_hop_date = group.loc[j, "START_DATE"]   # Record first hop
                    
                    if next_st>first_hop_date:   # Next hop 
                        if (second_hop_date == 0) or (second_hop_date == next_st):
                            update_info = {
                                'current_ONET': current_ONET,  # The ONET code of the current job experience
                                'next_ONET': next_ONET,  # The ONET code of the next or overlapping job
                                'update_value': 1  # Assuming a simple count increment for each hop
                                }
                            updates2.append(update_info)

                            second_hop_date = next_st   # Record the start date of the second hop 
    #print(updates2)
    return pd.DataFrame(updates2)

In [10]:
result2 = ddf.groupby('ID').apply(multihopcount, meta={'current_ONET': 'object', 'next_ONET': 'object', 'update_value': 'int'})

In [11]:
aggregated_ddf2 = result2.groupby(['current_ONET', 'next_ONET']).agg({'update_value': 'sum'}).reset_index()


In [13]:
aggregated_ddf21 = aggregated_ddf2.compute()

In [14]:
aggregated_ddf21

,current_ONET,next_ONET,update_value
0,21-1022.00,13-2052.00,30.0
1,11-9081.00,11-9039.00,5.0
2,41-9099.00,13-1071.00,1518.0
3,13-1082.00,11-1021.00,21446.0
4,41-2031.00,25-9044.00,2841.0
...,...,...,...
11095,51-8091.00,51-9032.00,1.0
11096,49-3043.00,21-1011.00,1.0
11097,31-9096.00,13-1051.00,1.0
11098,13-2023.00,19-4043.00,1.0


In [8]:
job_hop_count_df = pd.read_csv("data/raw/job_hop_count_empty.csv",index_col=0)
import numpy as np

In [16]:
aggregated_ddf21.to_parquet("Transitions_v5_2hop.parquet")
aggregated_ddf21.to_csv("Transitions_v5_2hop.csv")

piv = aggregated_ddf21.pivot_table(index='current_ONET', columns='next_ONET', values='update_value', fill_value=0)
all_labels = piv.index.union(job_hop_count_df.index)
pivot_table = piv.reindex(index=all_labels, columns=all_labels, fill_value=0)
pivot_table.to_csv("Transition_Matrix_v5_2hop.csv")

trans_prob = pivot_table.div(pivot_table.sum(axis = 1), axis=0).replace(np.nan, 0)
trans_prob.to_csv("Transition_Probab_Matrix_v5_2hop.csv")

In [17]:
trans_prob_backward = pivot_table.div(pivot_table.sum(axis = 0), axis=1).replace(np.nan, 0)

In [18]:
trans_prob_backward.to_csv("Bkwd_Transition_Probab_Matrix_v5_2hop.csv")

In [19]:
trans_prob

,11-1011.00,11-1011.03,11-1021.00,11-1031.00,11-2011.00,11-2021.00,11-2022.00,11-2032.00,11-2033.00,11-3012.00,...,55-3011.00,55-3012.00,55-3013.00,55-3014.00,55-3015.00,55-3016.00,55-3018.00,55-3019.00,99-9990,99-9999.00
11-1011.00,0.195514,0.000046,0.070093,0.000305,0.001701,0.037356,0.038719,0.017600,0.000184,0.001514,...,0.000000,0.000002,0.000000,0.000003,0.000013,0.000030,0.0,0.000033,0.036222,0.018220
11-1011.03,0.092873,0.025918,0.053996,0.002160,0.000000,0.049676,0.023758,0.036717,0.000000,0.002160,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.021598,0.015119
11-1021.00,0.091790,0.000039,0.159007,0.000104,0.001322,0.044269,0.064324,0.008361,0.000095,0.002086,...,0.000000,0.000004,0.000000,0.000006,0.000020,0.000023,0.0,0.000032,0.018489,0.010645
11-1031.00,0.112735,0.000232,0.028068,0.002552,0.002320,0.017165,0.019021,0.032707,0.000232,0.001856,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.087915,0.023660
11-2011.00,0.080483,0.000051,0.045828,0.000203,0.029518,0.157380,0.064007,0.045879,0.000508,0.001030,...,0.000000,0.000000,0.000000,0.000000,0.000025,0.000013,0.0,0.000000,0.021509,0.010678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55-3016.00,0.039946,0.000000,0.045663,0.000071,0.000388,0.012669,0.022020,0.003247,0.000071,0.000776,...,0.000000,0.000000,0.000106,0.000071,0.000071,0.002117,0.0,0.000282,0.032077,0.011822
55-3018.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
55-3019.00,0.040954,0.000000,0.046232,0.000000,0.000633,0.015411,0.019633,0.004433,0.000000,0.001900,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000211,0.0,0.002111,0.029343,0.013088
99-9990,0.027002,0.000074,0.020719,0.000090,0.002770,0.037171,0.015679,0.013201,0.000130,0.000675,...,0.000000,0.000002,0.000000,0.000004,0.000011,0.000048,0.0,0.000016,0.068994,0.023978


### Three Hops

In [ ]:
def multihopcount3(group):
    # Update list
    updates2 = []

    group = group.reset_index(drop=True)
    
    #display(group)

    # Go through the experiences of each ID 
    for i in range(len(group) - 1):
        # Extract current job info
        current_st = group.loc[i, "START_DATE"]
        current_end = group.loc[i, "END_DATE"]
        current_ONET = group.loc[i, "ONET"]
        
        if current_st == current_end: 
            first_hop_date = 0 
            second_hop_date = 0 
            third_hop_date = 0

            for j in range(i + 1, len(group)):
                next_st = group.loc[j, "START_DATE"]
                next_end = group.loc[j, "END_DATE"]
                next_ONET = group.loc[j, "ONET"]

                # First transition: Skip
                if next_st > current_end:  # No overlaps 
                    if first_hop_date == 0: 
                        first_hop_date = group.loc[j, "START_DATE"]   # Record first hop, but skip this count 
                    
                    # Second transition: Skip
                    if next_st>first_hop_date:
                        if second_hop_date == 0:    
                            second_hop_date = group.loc[j, "START_DATE"]   # Record second hop, but skip this count 
                        # Third transition: Count 
                        if next_st>second_hop_date:   # Next hop     
                            if (third_hop_date == 0) or (third_hop_date == next_st):
                                update_info = {
                                    'current_ONET': current_ONET,  # The ONET code of the current job experience
                                    'next_ONET': next_ONET,  # The ONET code of the next or overlapping job
                                    'update_value': 1  # Assuming a simple count increment for each hop
                                    }
                                updates2.append(update_info)

                                third_hop_date = next_st   # Record the start date of the second hop 


        elif current_st != current_end: 
            first_hop_date = 0 
            second_hop_date = 0 
            third_hop_date = 0

            for j in range(i + 1, len(group)):
                next_st = group.loc[j, "START_DATE"]
                next_end = group.loc[j, "END_DATE"]
                next_ONET = group.loc[j, "ONET"]

                # First transition: Skip
                if next_st >= current_end:  # No overlaps 
                    if first_hop_date == 0: 
                        first_hop_date = group.loc[j, "START_DATE"]   # Record first hop, but skip this count 
                    
                    # Second transition: Skip
                    if next_st>first_hop_date:
                        if second_hop_date == 0:    
                            second_hop_date = group.loc[j, "START_DATE"]   # Record second hop, but skip this count 
                        # Third transition: Count 
                        if next_st>second_hop_date:   # Next hop     
                            if (third_hop_date == 0) or (third_hop_date == next_st):
                                update_info = {
                                    'current_ONET': current_ONET,  # The ONET code of the current job experience
                                    'next_ONET': next_ONET,  # The ONET code of the next or overlapping job
                                    'update_value': 1  # Assuming a simple count increment for each hop
                                    }
                                updates2.append(update_info)

                                third_hop_date = next_st   # Record the start date of the second hop 
                            
    #print(updates2)
    return pd.DataFrame(updates2)

In [ ]:
result3 = ddf.groupby('ID').apply(multihopcount3, meta={'current_ONET': 'object', 'next_ONET': 'object', 'update_value': 'int'})

In [ ]:
aggregated_ddf3 = result3.groupby(['current_ONET', 'next_ONET']).agg({'update_value': 'sum'}).reset_index()
aggregated_ddf31 = aggregated_ddf3.compute()

In [ ]:
aggregated_ddf31

,current_ONET,next_ONET,update_value
0,17-2112.00,13-1199.00,1486.0
1,11-1021.00,11-3031.00,13751.0
2,11-1021.00,11-1011.00,20295.0
3,11-2011.00,41-9011.00,23.0
4,43-4051.00,13-1071.00,8149.0
...,...,...,...
42275,19-3051.00,53-3099.00,1.0
42276,51-3023.00,53-6032.00,1.0
42277,17-2111.00,47-2211.00,1.0
42278,39-2011.00,27-4014.00,1.0


In [ ]:
aggregated_ddf31.to_parquet("Transitions_v5_3hop.parquet")
aggregated_ddf31.to_csv("Transitions_v5_3hop.csv")

piv3 = aggregated_ddf31.pivot_table(index='current_ONET', columns='next_ONET', values='update_value', fill_value=0)
all_labels3 = piv3.index.union(job_hop_count_df.index)
pivot_table3 = piv3.reindex(index=all_labels3, columns=all_labels3, fill_value=0)
pivot_table3.to_csv("Transition_Matrix_v5_3hop.csv")

trans_prob3 = pivot_table3.div(pivot_table3.sum(axis = 1), axis=0).replace(np.nan, 0)
trans_prob3.to_csv("Transition_Probab_Matrix_v5_3hop.csv")

In [ ]:
trans_prob_backward3 = pivot_table3.div(pivot_table3.sum(axis = 0), axis=1).replace(np.nan, 0)
trans_prob_backward3.to_csv("Bkwd_Transition_Probab_Matrix_v5_3hop.csv")